In [1]:
import json
import pandas as pd
import glob
from maleo.wizard import Wizard
from tqdm import tqdm

In [2]:
def read_json(path):
    with open(path, 'r') as file:
        return json.load(file)

In [3]:
def get_all_hashtag(list_tweets):   
    all_hashtag = []
    
    wiz = Wizard()
    twt_hashtag = wiz.get_hashtag(pd.Series(list_tweets))['Hashtag']
    n_twt_use_hashtag = len(twt_hashtag)
    
    for i in twt_hashtag:
        all_hashtag += i
    return n_twt_use_hashtag, all_hashtag

In [4]:
data_path = '../data/data_7200/*.json'
profile_data_path = '../data/profile_id.json'

screen_name == username

Data pada key "tweets" terdiri atas independent & dependent tweet
 - Independent tweet = tweet yang dibuat sendiri (inspirasi sendiri)
 - Dependent tweet = tweet yang mengutip/quote tweet org lain (quoted tweet)

In [5]:
profile_id = read_json(profile_data_path)
username_desc = {user['screen_name']:(user['name'], user['description']) for user in profile_id}

In [ ]:
raw_features = []

for data in tqdm(glob.glob(data_path)[:5]):
    list_tweets = []
    list_quoted_tweets = []
    
    user_data = read_json(data)
    
    for twt in user_data['tweets']:
        list_tweets.append(twt['full_text'])
        try:
            list_quoted_tweets.append(twt['quoted_status']['full_text'])
        except:
            pass
        
    n_twt_use_hashtag, all_hashtag = get_all_hashtag(list_tweets)
    
    if n_twt_use_hashtag != 0:
        ratio = (n_twt_use_hashtag/len(list_tweets))
    else:
        ratio = 0
    
    username = data.split('/')[-1][:-5]
    if not username.startswith('@'):
        desc = username_desc.get(username)[1]
    else:
        desc = user_data['description']
        
    feat = {'username': username, 
            'name': username_desc.get(username)[0],
            'desc': desc,
            'tweets': list_tweets,
            'quoted_tweets': list_quoted_tweets,
            'n_tweet': len(list_tweets),
            'hashtag': all_hashtag,
            'n_tweet_use_hashtag': n_twt_use_hashtag,
            'ratio_tweets_use_hashtag': ratio}
    raw_features.append(feat)

In [ ]:
raw_features[0]

## Get media and url link

In [6]:
tes = read_json('../data/data_7200/@TsamaraDKI.json')

In [78]:
def get_media_and_url(data):
    media_type = None
    url_link = None
    
    if 'quoted_status' not in data:
        try:
            media_type = data['extended_entities']['media'][0]['type']
        except:
            pass
        if media_type != 'photo' and data['entities']['urls'] != []:
            url_link = data['entities']['urls'][0]['expanded_url']
    return media_type, url_link

In [79]:
def get_youtube_title(url_link):
    import urllib.request
    import json
    import urllib
    
    VideoID = url_link.split('/')[-1] 

    params = {"format": "json", "url": "https://www.youtube.com/watch?v=%s" % VideoID}
    url = "https://www.youtube.com/oembed"
    query_string = urllib.parse.urlencode(params)
    url = url + "?" + query_string

    with urllib.request.urlopen(url) as response:
        response_text = response.read()
        data = json.loads(response_text.decode())
        return data['title']

In [114]:
def coba(data):
    media_type, url_link = get_media_and_url(data)
    
    if url_link == None:
        title = None
    elif url_link.split('/')[-2] == 'youtu.be':
        title = get_youtube_title(url_link)
    else:
        title = url_link.split('/')[-1].split('?')[0].split('.')[0]
    return media_type, title

In [132]:
coba(tes['tweets'][21])

(None,
 'psi-hanura-pkpi-berkarya-hingga-perindo-tolak-kenaikan-pt-arogansi-kekuasaan-1tZ5TNVvJ2u')